In [1]:
import pandas as pd
import numpy as np
import concurrent.futures as cf
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import nltk
import pickle
from glob import glob
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from collections import Counter
import altair as alt
from scipy import stats
# !conda install -c conda-forge kneed -y
from kneed import KneeLocator

pd.set_option('display.max_columns', None)

%matplotlib inline

In [2]:
path_in = '/Users/ramonperez/Dropbox/Burning Glass/Data/random_data/'
path_out  = '/Users/ramonperez/Dropbox/Burning Glass/Analysis/unsupervised_approach/'
models = path_in + 'models/kmeans/'

In [3]:
%%time

sample_name = 'km_500and1000k_clean_dataset2.csv'

df = pd.read_csv(path_in + sample_name, low_memory=False)
df.head()

CPU times: user 44.7 s, sys: 1min 23s, total: 2min 8s
Wall time: 3min 30s


,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience,titles_uni_500,titles_bi_500,titles_uni_1000,titles_bi_1000,dist__bi_1000,dist__uni_500,dist__bi_500,dist__uni_1000,dist__bi_1000_dummy,dist__uni_500_dummy,dist__bi_500_dummy,dist__uni_1000_dummy,clean_text,charac_text_len,len_job_text
0,en,Allentown,PA,Advertising Account Executive Key Accounts,2016-07-08,38087992061,40.6023,-75.4707,18101,Lehigh,NaN,MT421090,10900: Metropolitan Statistical Area,Morning Call,Advertising Account Executive,41301100,NaN,0,Job Board,Marketing and Public Relations: Advertising;Sp...,{'Advertising': 'Marketing and Public Relation...,False,Bachelor's,Bachelor's,NaN,2.0,NaN,41-3011.00,NaN,NaN,NaN,NaN,fulltime,permanent,1-6,mid,16,Advertising Account Executive,16.0,16.0,Advertising Account Executive,At least 2-4 years|2 to 3 years,156,56,995,286,3631.16,1371.38,1783.33,2770.52,0,0,0,0,Advertising Account Executive (Key Accounts) T...,4633,684
1,en,Hanover,NJ,Crop Protection Sales Specialist For Bayer Cro...,2018-06-14,38379581429,40.8218,-74.4220,07981,Morris,35084.0,DV343508|MT363562,35620: Metropolitan Statistical Area|408: Comb...,Bayer Corporation,Sales Specialist,41401200,NaN,0,Company,Finance: Budget Management;Specialized Skills|...,{'Budgeting': 'Finance: Budget Management;Spec...,False,NaN,NaN,NaN,NaN,325412.0,41-4011.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sales Specialist,NaN,NaN,Sales Representative,NaN,0,486,581,196,4453.73,1814.80,2200.47,3651.45,0,0,0,0,. Crop Protection Sales Specialist for Ba...,3266,435
2,en,Milwaukee,WI,Program And Policy Chief Performance Improveme...,2017-03-27,38202025999,43.0343,-87.9151,53201,Milwaukee,NaN,MT553334,33340: Metropolitan Statistical Area,Wisdot,NaN,11919900,NaN,0,Job Board,Finance: Budget Management;Specialized Skills|...,{'Budgeting': 'Finance: Budget Management;Spec...,False,NaN,NaN,NaN,NaN,NaN,11-9199.00,90209.0,43.37,62400.0,30.0,fulltime,permanent,NaN,NaN,NaN,Chief Program Performance Manager,NaN,NaN,Program Officer,NaN,313,158,58,607,3851.57,1508.76,1898.18,3053.94,0,0,0,0,In partnership with Hispanic Professionals of ...,5141,740
3,en,West Caldwell,NJ,Machine Operator/Nutraceutical,2020-07-17,38815921335,40.8069,-74.2778,07006,Essex,35084.0,DV343508|MT363562,35620: Metropolitan Statistical Area|408: Comb...,Vitaquest,Machine Operator,51919900,NaN,0,Job Board,Science and Research: Biopharmaceutical Manufa...,{'Batch Records': 'Science and Research: Bioph...,False,Higher Secondary Certificate,Higher Secondary Certificate,NaN,1.0,NaN,51-9012.92,NaN,NaN,NaN,NaN,fulltime,permanent,0-1,low,12,Machine Operator,NaN,12.0,Manufacturing Machine Operator,Minimum of 1 year,212,227,653,642,2668.94,1062.01,1292.53,2162.32,0,0,0,0,Machine Operator / Nutraceutical Vitaquest - W...,3417,516
4,en,Houston,TX,Sales Process Coordinator - Midmarket,2018-04-25,38353595694,30.0683,-95.2163,77325,Harris,NaN,MT482642,26420: Metropolitan Statistical Area,Recruiting Agency,Sales Coordinator,41401200,Insperity,0,Job intermediary,Business: Business Consulting;Specialized Skil...,{'Business Consulting': 'Business: Business Co...,False,Higher Secondary Certificate,Higher Secondary Certificate,NaN,2.0,NaN,41-4011.00,NaN,NaN,NaN,NaN,NaN,NaN,1-6,mid,12,Sales Coordinator,NaN,12.0,Sales Coordinator,One to two-years,315,232,327,756,3160.85,1260.74,1544.18,2553.99,0,0,0,0,Sales Process Coordinator - MidMarket - Job Ti...,3045,452


In [9]:
df.groupby('titles_uni_500')['titles_uni_500'].count().sort_values(ascending=False).head(15)

titles_uni_500
9      49051
395    15671
21     12935
56     12325
402    12140
157    10486
315    10361
193     9581
80      9382
185     9140
198     7770
2       7758
5       6516
398     5769
48      5252
Name: titles_uni_500, dtype: int64

In [10]:
df.loc[df['titles_uni_500'] == 198, 'CleanJobTitle'].head(25)

114                                   Project Assistant I
135           Executive Assistant, International Law Firm
266             Executive Assistant Fyi Television Inc In
330                          Patient Scheduling Assistant
505                             Senior Assistant Database
520              Acm Rsch Assistant II Bench Nanomedicine
566                                   Data Desk Assistant
619              Federal Work Study - Equipment Assistant
716                         Ltd Appt - Forestry Assistant
724                               Dental Assistant -Pt/Wk
792     Assistant Vice President For Governmental Affa...
801                                        Dnfb Assistant
1180                          Accounting Assistant Casino
1405                Visitor Use Assistant Fee Collections
1702                       Production/Warehouse Assistant
1748                                Dental Assistant - We
1810             Executive Assistant - Blue Zones Project
1844          

In [11]:
df.loc[df['titles_uni_500'] == 2, 'CleanJobTitle'].head(25)

7                        Urgent Care Director/Montana4669
203                                    Telephony Director
227                           Director - Decision Support
491                Imit Program Director, Imit Grant/Sria
509                          Director Portfolio Reporting
642     Director - Digital Forensics And Incident Resp...
644                       Store Director - J Crew Factory
650                          Academic Director For Design
779                 Director, People Analytics & Insights
861                                Director, Front Office
942                                  Director, Laboratory
1013         Director, Advanced Analytic Strategy - 18415
1127        Director, Speech Privacy & Technician Project
1184           Higher Education Campus President/Director
1189                             Director, Member Support
1222                       Architectural Project Director
1257             Director, On Marq Autism Support Program
1382          

In [12]:
df.loc[(df['titles_uni_500'] == 2) &
        (df['dist__uni_500_dummy'] == 1), 'CleanJobTitle'].head(25)

203                          Telephony Director
942                        Director, Laboratory
1492             Deputy Director - Aed001572Aho
1616                          Director I Claims
2193                        Director - Benefits
4566         Stereoscopic Technical Director, R
5257                      Salesmanager/Director
6268                     Director - 3 Positions
7813                       Director, Valuations
8911                         Pediatric Director
9037                               Mds Director
10191             Active Director Administartor
10865                Director, Mamp A Execution
13415                Rhfa A/O Division Director
15320                           Director/Editor
16578                         Forensic Director
16702                           Trauma Director
17744                  Director Home Health3553
20170    Histocompatibility Laboratory Director
21059                 Stage Director, Ruddigore
21154                     Purchasing Dir

In [13]:
df.groupby('occu_text')['titles_uni_500'].count().sort_values(ascending=False).head(25)

occu_text
Registered Nurses                                                                        33036
Computer Occupations, All Other                                                          31332
Computer Programmers                                                                     29942
Sales Representatives, Wholesale and Manufacturing, Technical and Scientific Products    17509
Retail Salespersons                                                                      16325
Customer Service Representatives                                                         13079
Medical and Health Services Managers                                                     12871
First-Line Supervisors of Retail Sales Workers                                           11148
Accountants and Auditors                                                                 11090
Secretaries and Administrative Assistants, Except Legal, Medical, and Executive           9814
Sales Managers                          

In [14]:
df.drop('occu_code', axis=1, inplace=True)

In [15]:
# !conda install -c pyviz datashader -y

In [16]:
df[df['titles_uni_500'] == 2].groupby('occu_text')['titles_uni_500'].count().sort_values(ascending=False).head(25)

occu_text
Medical and Health Services Managers                                                     1047
Managers, All Other                                                                       463
Education Administrators, Postsecondary                                                   320
Computer Occupations, All Other                                                           265
Computer and Information Systems Managers                                                 243
Sales Managers                                                                            201
Art Directors                                                                             129
Social and Community Service Managers                                                      95
Sales Representatives, Wholesale and Manufacturing, Technical and Scientific Products      85
Natural Sciences Managers                                                                  79
Recreation Workers                                

In [17]:
%%time

for title in df.columns:
    if 'titles' in title:
        df[title] = df[title].astype(np.int16)
    elif 'dummy' in title:
        df[title] = df[title].astype(np.bool_)
    elif 'dist' in title and 'dummy' not in title:
        df[title] = df[title].astype(np.float16)

CPU times: user 144 ms, sys: 81.4 ms, total: 226 ms
Wall time: 230 ms


In [18]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709748 entries, 0 to 709747
Data columns (total 58 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Language                          709748 non-null  object 
 1   CanonCity                         700070 non-null  object 
 2   CanonState                        709647 non-null  object 
 3   CleanJobTitle                     709748 non-null  object 
 4   JobDate                           709748 non-null  object 
 5   JobID                             709748 non-null  int64  
 6   Latitude                          700103 non-null  float64
 7   Longitude                         700103 non-null  float64
 8   CanonPostalCode                   697904 non-null  object 
 9   CanonCounty                       699770 non-null  object 
 10  DivisionCode                      250056 non-null  float64
 11  LMA                               697944 non-null  o

In [19]:
df.tail()

,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience,titles_uni_500,titles_bi_500,titles_uni_1000,titles_bi_1000,dist__bi_1000,dist__uni_500,dist__bi_500,dist__uni_1000,dist__bi_1000_dummy,dist__uni_500_dummy,dist__bi_500_dummy,dist__uni_1000_dummy,clean_text,charac_text_len,len_job_text,occu_text
709743,en,Tampa,FL,Rcm - Accounts Receivable Specialist,2018-01-09,38316465836,27.9827,-82.3402,33601,Hillsborough,NaN,MT124530,45300: Metropolitan Statistical Area,Recruiting Agency,Accounts Receivable Specialist,43-303100,Vaco Staffing,0,Job intermediary,Finance: Accounts Payable and Receivable;Speci...,{'Accounts Payable / Accounts Receivable': 'Fi...,False,Higher Secondary Certificate,Associate's|General Equivalency Degree|Higher ...,NaN,1.0,6221.0,433031,NaN,NaN,NaN,NaN,NaN,NaN,1-6,mid,14|12,Accounts Receivable Specialist,14.0,12.0,Accounts Receivable / Payable Specialist,1 to 3 years|1-3 years,395,127,150,234,3180.0,1273.0,1553.0,2574.0,False,False,False,False,RCM - A/R Specialist $19/HR -Vaco Staffing Ta...,1234,167,"Bookkeeping, Accounting, and Auditing Clerks"
709744,en,Durango,CO,Rcis/Registered Cardiovascular Invasive Specia...,2017-03-02,38189722337,37.3268,-107.8310,81301,La Plata,NaN,MC082042,20420: Micropolitan Statistical Area,Centura Health,NaN,29203100,NaN,0,Job Board,Health Care: Emergency and Intensive Care;Spec...,{'Advanced Cardiac Life Support (ACLS)': 'Heal...,False,Bachelor's,Bachelor's,510901,1.0,622110.0,292031,NaN,NaN,NaN,NaN,fulltime,permanent,0-1,low,16,Registered Cardiovascular Specialist,NaN,16.0,Cardiovascular Technician / Technologist,One year|6 months,395,4,110,813,3318.0,1359.0,1624.0,2746.0,False,False,False,False,RCIS/Registered Cardiovascular Invasive Specia...,3921,531,Cardiovascular Technologists and Technicians
709745,en,Frederick,MD,"Process Systems Software Test Engineer - , Doc...",2011-10-04,6981341,39.4944,-77.4535,21702,Frederick,43524.0,DV241364|MT114790,47900: Metropolitan Statistical Area|548: Comb...,U7 Equipment,Software Test Engineer,15-119902,NaN,0,Job Board,Engineering: Engineering Software;Specialized ...,{'LabVIEW': 'Engineering: Engineering Software...,False,Bachelor's,Bachelor of Science|Bachelor's|Doctor of Medicine,140101|141001|141901,5.0,62.0,151133,100000.0,48.08,75000.0,36.06,NaN,NaN,6+,high,16|21,Software Test Engineer,21.0,16.0,Systems Engineer,5-10 years,495,163,919,895,4356.0,1604.0,2150.0,3230.0,False,False,False,False,| | | Employers &raquo; Process Systems So...,3839,557,"Software Developers, Systems Software"
709746,en,New York,NY,Expert Programmer,2019-04-11,38516931242,40.6940,-73.9898,11201,Kings,35614.0,DV363564|MT363562,35620: Metropolitan Statistical Area|408: Comb...,Logicsoft,NaN,15113100,NaN,0,Job Board,NaN,{},False,NaN,NaN,NaN,8.0,NaN,151131,176800.0,85.00,176800.0,85.00,fulltime,temporary,6+,high,NaN,Programmer,NaN,NaN,Computer Programmer,8 years,340,274,414,170,2470.0,1063.0,1189.0,2166.0,True,False,True,False,Logicsoft Logo M204 expert programmer Applied ...,715,110,Computer Programmers
709747,en,Cheyenne,WY,Qsr Assistant Manager,2017-01-05,38162819143,41.1245,-104.8400,82001,Laramie,NaN,MT561694,16940: Metropolitan Statistical Area,Kroger Company,Assistant Manager,41101100,NaN,0,Job Board,Finance: Budget Management;Specialized Skills|...,{'Budgeting': 'Finance: Budget Management;Spec...,False,Higher Secondary Certificate,NaN,NaN,2.0,445110.0,411011,NaN,NaN,NaN,NaN,fulltime,permanent,1-6,mid,12,Assistant Mana

In [20]:
sample_name

'km_500and1000k_clean_dataset2.csv'

In [21]:
%%time

df.to_csv(path_in + 'km_500and1000k_clean_dataset2.csv', index=False)

CPU times: user 1min 27s, sys: 8.29 s, total: 1min 35s
Wall time: 1min 42s
